In [3]:
# Importing Necessary Libraries
import numpy as np
import pandas as pd
import shutil
import cv2
import os

In [4]:
# The data needs to be preprocessed, we need to make new folders for each species and 
# Shift the corresponding image to the respcetive folder
dataset = pd.read_csv('dataset/train.csv')
file_names = list(dataset['image_id'].values)
img_labels = list(dataset['breed'].values)

In [5]:
lst = list(np.unique(dataset['breed'].values))

In [8]:
source = os.getcwd()

for new_path in lst:
    if not os.path.exists("." + new_path):
        os.makedirs(new_path)

In [10]:
folders = lst.copy()

for f in range(len(file_names)):
    current_img = file_names[f]
    current_label = img_labels[f]
    shutil.move("dataset//train//"+current_img+".jpg", current_label+"//"+current_img+".jpg")

In [9]:
for label in os.listdir("data"):
    for image in os.listdir("data//"+label):
        img = cv2.resize(cv2.imread("data//"+label+"//"+image),(128,128))
        cv2.imwrite("data//"+label+"//"+image,img)

In [ ]:
# From here we start building the image classifier.
# The preprocessing will allow us to use data flow from directry function so as to reduce time consumption 
# And enable data augmentation
# Each image is resized to fit 128x128 and retained all the 3 layers for each image

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import time

In [7]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        featurewise_center=True,
        featurewise_std_normalization=True,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        vertical_flip=True,
        horizontal_flip=True,
        zca_epsilon=1e-06,
        brightness_range = (0.1,0.3))

In [8]:
gen = train_datagen.flow_from_directory(
        'data',
        target_size=(128, 128),
        batch_size=128,
        class_mode='categorical',
        color_mode="rgb",
        shuffle=True,
    seed=None,)

Found 5508 images belonging to 35 classes.


In [2]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img 
from keras.models import Sequential 
from keras import optimizers
from keras.preprocessing import image
from keras.layers import  Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras import applications 
from keras.utils.np_utils import to_categorical 
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
%matplotlib inline

In [17]:
# define the model and start building the cnn
model = Sequential()

In [18]:
# Conv2D( number_of_filters , kernal_size , input_shape(add this parameter just for the input conv layer))
model.add(Conv2D(64 , (3,3) , input_shape = (128,128,3) ))
# define the activaion function for this layer
model.add(Activation('relu'))
# define the pooling for this layer
model.add(MaxPooling2D(pool_size= (2,2)))

In [19]:
model.add(Conv2D(64 , (3,3) ))
# define the activaion function for this layer
model.add(Activation('relu'))
# define the pooling for this layer
model.add(MaxPooling2D(pool_size= (2,2)))

In [20]:
model.add(Conv2D(32 , (3,3) ))
# define the activaion function for this layer
model.add(Activation('relu'))
# define the pooling for this layer
model.add(MaxPooling2D(pool_size= (2,2)))

In [21]:
model.add(Flatten())
model.add(Dense(64))

In [22]:
model.add(Dense(35))
model.add(Activation('softmax'))

In [23]:
model.compile(loss = 'binary_crossentropy', 
                optimizer = 'adam', 
                metrics = ['accuracy'])

In [ ]:
start = time.time()
model.fit_generator(
    gen,
    steps_per_epoch=5,
    epochs=10)
end = time.time()
print('Processing time:',(end - start)/60)
model.save_weights('model_baseline.h5')

Epoch 1/10
